In [1]:
import os

import numpy as np
from models.attention import multihead_model
from models.selfattention import selfattention_model, selfattention_model_modified
from models.baseline_model import baseline, baseline_residual, baseline_modified
from keras.callbacks import ModelCheckpoint, EarlyStopping
from data_utility.file_utility import FileUtility
from data_utility.labeling_utility import LabelingData
import itertools
from data_utility.feedgenerator import train_batch_generator_408, validation_batch_generator_408


os.environ["CUDA_VISIBLE_DEVICES"] = '1'

Using TensorFlow backend.


In [2]:
def run_baseline(epochs=10, setting_name='basemodel', gpu='1', train_batch_size=64,
                 test_batch_size=100, patience=10, features_to_use=['onehot', 'sequence_profile'], convs=[3, 5, 7], dense_size=200, lstm_size=400,use_CRF=False):
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu

    # read files
    train_file = '../DeepSeq2Sec/data/s8_all_features/train.txt'
    test_file = '../DeepSeq2Sec/data/s8_all_features/test.txt'
    LD = LabelingData(train_file, test_file)
    train_lengths = [int(j) for j in FileUtility.load_list('/'.join(train_file.split('/')[0:-1]) + '/train_length.txt')]
    test_lengths = [int(i) for i in FileUtility.load_list('/'.join(test_file.split('/')[0:-1]) + '/test_length.txt')]

    # model
    model, params = baseline_residual(LD.n_classes, features_to_use=features_to_use, convs=convs,
                             dense_size=dense_size, lstm_size=lstm_size,use_CRF=use_CRF)

    # output directory
    FileUtility.ensure_dir('baseline_modifications/' + setting_name + params + '/')

    # save model
    with open('baseline_modifications/' + setting_name + params + '/' + 'config.txt', 'w') as fh:
        model.summary(print_fn=lambda x: fh.write(x + '\n'))

    # check points
    filepath = 'baseline_modifications/' + setting_name + params + "/weights-improvement-{epoch:02d}-{weighted_acc:.3f}-{val_weighted_acc:.3f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_weighted_acc', verbose=1, save_best_only=True, mode='max',
                                 period=1)
    earlystopping = EarlyStopping(monitor='val_weighted_acc', min_delta=0, patience=patience, verbose=0, mode='max',
                                  baseline=None)
    callbacks_list = [checkpoint, earlystopping]

    # calculate the sizes
    steps_per_epoch = len(train_lengths) / train_batch_size if len(train_lengths) % train_batch_size == 0 else int(
        len(train_lengths) / train_batch_size) + 1
    validation_steps = int(len(test_lengths) / test_batch_size) if len(test_lengths) % test_batch_size == 0 else int(
        len(test_lengths) / test_batch_size) + 1

    # feed model
    h = model.fit_generator(train_batch_generator_408(train_batch_size), steps_per_epoch=steps_per_epoch,
                            validation_data=validation_batch_generator_408(test_batch_size),
                            validation_steps=validation_steps,
                            shuffle=False, epochs=epochs, verbose=1, callbacks=callbacks_list)

    # save the history
    FileUtility.save_obj('baseline_modifications/' + setting_name + params + '/history', h.history)


In [4]:
run_baseline(epochs=300, setting_name='residual2_', gpu='0', train_batch_size=32,
                 test_batch_size=100, patience=10, features_to_use=['onehot', 'sequence_profile'], convs=[3,5,7,11,21], dense_size=1000, lstm_size=1000,use_CRF=False)

Labeling utility object created..
Training y encoded shape is  (5534, 700)
Maximum sequence length is 700
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 408)    0                                            
__________________________________________________________________________________________________
onehot (Lambda)                 (None, None, 21)     0           input_2[0][0]                    
__________________________________________________________________________________________________
sequenceprofile (Lambda)        (None, None, 21)     0           input_2[0][0]                    
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, None, 42)     0           onehot[0][0]                     
   

Epoch 7/300
173/173 [==============================] - 29s 170ms/step - loss: 0.8212 - weighted_acc: 0.7188 - val_loss: 0.9305 - val_weighted_acc: 0.6754

Epoch 00007: val_weighted_acc improved from 0.66981 to 0.67542, saving model to baseline_modifications/residual2_model#onehot#sequence_profile@conv3_5_7_11_21@dense_1000@lstm1000@droplstm0.5@filtersize_32/weights-improvement-07-0.719-0.675.hdf5
Epoch 8/300
173/173 [==============================] - 30s 171ms/step - loss: 0.8108 - weighted_acc: 0.7219 - val_loss: 0.9309 - val_weighted_acc: 0.6745

Epoch 00008: val_weighted_acc did not improve from 0.67542
Epoch 9/300
173/173 [==============================] - 30s 171ms/step - loss: 0.8049 - weighted_acc: 0.7236 - val_loss: 0.9229 - val_weighted_acc: 0.6780

Epoch 00009: val_weighted_acc improved from 0.67542 to 0.67800, saving model to baseline_modifications/residual2_model#onehot#sequence_profile@conv3_5_7_11_21@dense_1000@lstm1000@droplstm0.5@filtersize_32/weights-improvement-09-0.7

Epoch 35/300
173/173 [==============================] - 29s 167ms/step - loss: 0.7072 - weighted_acc: 0.7578 - val_loss: 0.9038 - val_weighted_acc: 0.6887

Epoch 00035: val_weighted_acc improved from 0.68844 to 0.68869, saving model to baseline_modifications/residual2_model#onehot#sequence_profile@conv3_5_7_11_21@dense_1000@lstm1000@droplstm0.5@filtersize_32/weights-improvement-35-0.758-0.689.hdf5
Epoch 36/300
173/173 [==============================] - 29s 166ms/step - loss: 0.7035 - weighted_acc: 0.7595 - val_loss: 0.9115 - val_weighted_acc: 0.6855

Epoch 00036: val_weighted_acc did not improve from 0.68869
Epoch 37/300
173/173 [==============================] - 29s 166ms/step - loss: 0.7006 - weighted_acc: 0.7603 - val_loss: 0.9019 - val_weighted_acc: 0.6900

Epoch 00037: val_weighted_acc improved from 0.68869 to 0.68997, saving model to baseline_modifications/residual2_model#onehot#sequence_profile@conv3_5_7_11_21@dense_1000@lstm1000@droplstm0.5@filtersize_32/weights-improvement-37-

173/173 [==============================] - 29s 167ms/step - loss: 0.6189 - weighted_acc: 0.7905 - val_loss: 0.9143 - val_weighted_acc: 0.6925

Epoch 00067: val_weighted_acc did not improve from 0.69336
Epoch 68/300
173/173 [==============================] - 29s 166ms/step - loss: 0.6160 - weighted_acc: 0.7918 - val_loss: 0.9125 - val_weighted_acc: 0.6930

Epoch 00068: val_weighted_acc did not improve from 0.69336
Epoch 69/300
173/173 [==============================] - 29s 166ms/step - loss: 0.6135 - weighted_acc: 0.7926 - val_loss: 0.9168 - val_weighted_acc: 0.6926

Epoch 00069: val_weighted_acc did not improve from 0.69336
Epoch 70/300
173/173 [==============================] - 29s 167ms/step - loss: 0.6123 - weighted_acc: 0.7932 - val_loss: 0.9181 - val_weighted_acc: 0.6909

Epoch 00070: val_weighted_acc did not improve from 0.69336
Epoch 71/300
173/173 [==============================] - 29s 167ms/step - loss: 0.6097 - weighted_acc: 0.7938 - val_loss: 0.9167 - val_weighted_acc: 0.694